In [26]:
import numpy as np
import scipy.io.wavfile as wavfile
from scipy.signal import lfilter
import librosa
# def Chopper(Input, WindowLength, Window, HoppingSize):
#     """
#     Chopper windows the signal based on window length, shift percentage and
#     uses Hamming windowing technique.
#     """

#     # Number of samples to hop.
#     HoppingSamples = int(WindowLength * HoppingSize)

#     # Number of segments.
#     NumSegments = int(((len(Input) - WindowLength) / HoppingSamples) + 1)

#     # Index matrix which guides the signal through chopping process.
#     Index = (np.tile(np.arange(WindowLength), (NumSegments, 1)).T +
#              np.tile(np.arange(0, NumSegments * HoppingSamples, HoppingSamples), (WindowLength, 1)))

#     # Final window which multiplies with original signal to give pieces of it.
#     FinalWindow = np.tile(Window, (NumSegments, 1)).T

#     # Ta-da...
#     Output = Input[Index] * FinalWindow

#     return Output, NumSegments

def Chopper(input_signal, window_length,hopping_size):
    hopping_samples = window_length*hopping_size
    number_of_segments = int(np.fix(((len(input_signal)-window_length)/hopping_samples)+1))
    chopped_signal = []
    for i in range(number_of_segments):
        if (i+1)*window_length >= len(input_signal):
            filter_coeff = np.zeros(window_length)
            filter_coeff[:((i+1)*window_length)-len(input_signal)] = input_signal[(i*window_length):]
        else:
            filter_coeff = input_signal[i*window_length:(i+1)*window_length]
        chopped_signal.append(filter_coeff)
    return chopped_signal,len(chopped_signal)

# Setting up the variables before jumping into processing.
WinLenSec = 0.0025 # Window length in seconds.
HopPercent = 1 # percentage of hopping.
AROrder = 20 # Auto regressive filter order.
NumIter = 7

# Reading Input signal and creating noisy input as well.
Fs, Input = wavfile.read('input.wav')
Noise = np.random.normal(0, np.sqrt(0.01), size=Input.shape)
NoisyInput = Input + Noise
Time = np.arange(len(Input))/Fs

# Chopping session.
WinLenSamples = int(np.fix(WinLenSec * Fs))
Window = np.ones((WinLenSamples,1))
# ChoppedSignal, NumSegments = Chopper(NoisyInput, WinLenSamples, Window, HopPercent)
ChoppedSignal, NumSegments = Chopper(NoisyInput, WinLenSamples, HopPercent)

# Matrix initializations.
H = np.hstack((np.zeros((1,AROrder-1)), np.ones((1,1)))) # Measurement matrix.
R = np.var(Noise) # Variance of noise.

# FiltCoeff,Q = lfilter(np.hstack((1, -ChoppedSignal[0, :-1])), 1, ChoppedSignal[:, 0], 
#                         N=AROrder, return_signal=True) # Finding filter coefficients.

FiltCoeff = librosa.lpc(np.array(ChoppedSignal),order=AROrder)
Q = np.var(FiltCoeff,1).reshape((-1,1))

P = R * np.eye(AROrder) # Error covariance matrix.
Output = np.zeros(Input.shape) # Allocating memory for output signal.
Output[:AROrder] = NoisyInput[:AROrder] # Initializing output signal according to equation (13)
OutputP = NoisyInput[:AROrder]

# Iterators.
i = AROrder
j = AROrder

# Processing
for k in range(NumSegments):
    jStart = j
    OutputOld = OutputP
    
    for l in range(NumIter):
#         A = np.concatenate((np.zeros((AROrder-1, 1)), np.eye(AROrder-1), np.fliplr(-FiltCoeff[k, 1:].reshape(-1, 1))), axis=1)
        A = np.vstack((np.hstack((np.zeros((AROrder-1,1)),np.eye(AROrder-1))),np.flip(-1*FiltCoeff[k][1:])))
        for ii in range(i, WinLenSamples):
            OutputC = np.dot(A, OutputP).reshape((-1,1))
            Pc = np.dot(np.dot(A, P), A.T) + (np.dot(H.T, Q[k]) * H)
            K = np.dot(Pc, H.T)/((np.dot(H, np.dot(Pc, H.T)) + R)[0, 0])           
            OutputP = OutputC + K * (ChoppedSignal[k][ii] - (H@OutputC))
            Output[j-AROrder:j] = OutputP.T
            P = np.dot((np.eye(AROrder) - np.dot(K, H)), Pc)
            j += 1
        
        i = 1
        if l < NumIter - 1:
            j = jStart
            OutputP = OutputOld
        
        # update lpc on filtered signal
        for k in range(NumSegments):
            print(Output[k*WinLenSamples:(k+1)*WinLenSamples].shape)
            FiltCoeff[k,:], Q[k] = librosa.lpc(Output[k*WinLenSamples:(k+1)*WinLenSamples], order=AROrder)
    
# Convert the output to a column vector
Output = Output.reshape(-1, 1)


(55,)


ValueError: too many values to unpack (expected 2)